# SMART PREDICT NOTEBOOK 

## 1. MERGING CLEANED DATASETS
===============

### Input Datasets:

- **nasdaq_eps_merged**
  - Granularity: Daily  
  - Contains: Ticker, Earnings_Date, EPS_Estimate, EPS_Actual, Surprise(%), EPS_Surprise, and potentially macroeconomic indicators

- **merged_data_with_features**
  - Granularity: Daily  
  - Contains: Ticker, Date, Open, Close, Volume, and technical indicator columns

---

### Merge Strategy (Updated):

- Merging on an exact match between `Ticker` and `Date == Earnings_Date`.  
- EPS and macro data are joined only on the earnings release date.
- Converting both `Date` and `Earnings_Date` to a normalized date format before merging.
- Column names are standardized to ensure compatibility.


In [18]:
import pandas as pd

# Load data
features_df = pd.read_csv("merged_data_with_features.csv", parse_dates=["Date"])
eps_df = pd.read_csv("nasdaq_eps_merged.csv", parse_dates=["Date", "Earnings_Date"])
nasdaq_index_df = pd.read_csv("NASDAQ-100_cleaned.csv")

# Standardize and normalize date formats
features_df.rename(columns={"ticker": "Ticker"}, inplace=True)
features_df["Date"] = pd.to_datetime(features_df["Date"], errors="coerce").dt.normalize().dt.tz_localize(None)
eps_df["Earnings_Date"] = pd.to_datetime(eps_df["Earnings_Date"], errors="coerce").dt.normalize().dt.tz_localize(None)
eps_df["Date"] = pd.to_datetime(eps_df["Date"], errors="coerce").dt.normalize().dt.tz_localize(None)

# Drop duplicate EPS entries
eps_clean = eps_df.drop_duplicates(subset=["Ticker", "Earnings_Date"])[
    ["Ticker", "Earnings_Date", "EPS_Estimate", "EPS_Actual", "EPS_Surprise", "Surprise(%)"]
]

# Merge features with EPS data (Cartesian join on Ticker)
features_with_eps = features_df.merge(
    eps_clean,
    on="Ticker",
    how="left"
)

# Ensure datetime columns are tz-naive AFTER MERGE
features_with_eps["Earnings_Date"] = pd.to_datetime(features_with_eps["Earnings_Date"], errors="coerce").dt.tz_localize(None)
features_with_eps["Date"] = pd.to_datetime(features_with_eps["Date"], errors="coerce").dt.tz_localize(None)

# Keep only EPS rows where Earnings_Date is on or before feature Date
features_with_eps = features_with_eps[features_with_eps["Earnings_Date"] <= features_with_eps["Date"]]

# Keep most recent EPS record before each Date
features_with_eps.sort_values(by=["Ticker", "Date", "Earnings_Date"], inplace=True)
features_with_eps = features_with_eps.groupby(["Ticker", "Date"]).last().reset_index()

# Merge EPS info back into original features
final_df = pd.merge(
    features_df,
    features_with_eps[["Ticker", "Date", "EPS_Estimate", "EPS_Actual", "EPS_Surprise", "Surprise(%)"]],
    on=["Ticker", "Date"],
    how="left"
)

# Process NASDAQ-100 index data
nasdaq_index_df.columns = ["Open", "High", "Low", "Close", "Volume", "Dividends", "Stock_Splits"]
start_date = features_df["Date"].min()
nasdaq_index_df["Date"] = pd.date_range(start=start_date, periods=len(nasdaq_index_df), freq="B")
nasdaq_index_df["Ticker"] = "NASDAQ-100"

# Merge index data into final DataFrame
final_df = pd.merge(
    final_df,
    nasdaq_index_df,
    on=["Ticker", "Date"],
    how="left"
)

# Save to CSV
final_df.to_csv("final_dataset.csv", index=False)
final_df.head()



,Date,Open_x,High_x,Low_x,Close_x,Volume_x,transactions,Ticker,close_lag,volume_lag,...,EPS_Actual,EPS_Surprise,Surprise(%),Open_y,High_y,Low_y,Close_y,Volume_y,Dividends,Stock_Splits
0,2015-02-17,31.8725,32.2200,31.7300,31.9575,252516820.0,310744,AAPL,31.9575,252516820.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-02-18,31.9063,32.1950,31.8625,32.1788,179558052.0,226461,AAPL,31.9575,252516820.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015-02-19,32.1200,32.2575,32.0825,32.1125,149449524.0,199195,AAPL,32.1788,179558052.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015-02-20,32.1550,32.3750,32.0125,32.3738,195793676.0,236741,AAPL,32.1125,149449524.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015-02-23,32.5050,33.2500,32.4150,33.2500,283896440.0,340905,AAPL,32.3738,195793676.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
